In [19]:
import numpy as np

from astropy import units as u

import plotly.graph_objects as go

import lightkurve as lk

import itertools 

from wotan import slide_clip
from wotan import flatten
from wotan import t14

from transitleastsquares import cleaned_array
from transitleastsquares import transitleastsquares
from transitleastsquares import catalog_info
from detecta import detect_peaks

In [10]:
search_result_1 = lk.search_lightcurve("KIC 4840672")
lc_collection_1 = search_result_1[search_result_1.author == "Kepler"].download_all()

In [22]:
print(type(lc_collection_1), type(lc_collection_1[0]))

<class 'lightkurve.collections.LightCurveCollection'> <class 'lightkurve.lightcurve.KeplerLightCurve'>


In [20]:
lc_collection_1.stellar_properties=catalog_info(KIC_ID=int(lc_collection_1[0].meta['KEPLERID']))

In [59]:
class lightcurve_coll(lk.collections.LightCurveCollection):
    def __init__(self, lc_coll):
        super(lk.collections.LightCurveCollection, self).__init__(lc_coll)

        stellar_properties=catalog_info(KIC_ID=int(lc_coll[0].meta['KEPLERID']))
        self.mass=float(stellar_properties[1])
        self.radius=float(stellar_properties[4])
        
        
        stitched=self.stitch()
        quality_index = np.where(stitched['quality'] == 0)[0]
        stitched=stitched[quality_index]
        lc=stitched.flatten(window_length=901).remove_outliers(sigma_lower=np.Inf, sigma_upper=2)
        period_guess = np.linspace(1, 150, 20000)
        pg = lc.to_periodogram(method='bls', period=period_guess, frequency_factor=700)
        period = float(pg.period_at_max_power/u.d)
        tdur = t14(R_s=self.radius, M_s=self.mass, P=period, small_planet=True)
        window_leng=3*tdur
        
        
        self.wl=window_leng
        self.wl_period=period
      
            
        self.wl_lc=lc
        
    


In [60]:
test1=lightcurve_coll(lc_collection_1)

In [57]:
test1.mass
test1.radius

0.5960000157356262

In [58]:
test1.window_len(return_lc=False)

In [62]:
test1.wl_lc

time,flux,flux_err,quality,timecorr,centroid_col,centroid_row,cadenceno,sap_flux,sap_flux_err,sap_bkg,sap_bkg_err,pdcsap_flux,pdcsap_flux_err,sap_quality,psf_centr1,psf_centr1_err,psf_centr2,psf_centr2_err,mom_centr1,mom_centr1_err,mom_centr2,mom_centr2_err,pos_corr1,pos_corr2
,,,,d,pix,pix,,electron / s,electron / s,electron / s,electron / s,electron / s,electron / s,,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix
Time,float64,float64,int32,float32,float64,float64,int32,float32,float32,float32,float32,float32,float32,int32,float64,float32,float64,float32,float64,float32,float64,float32,float32,float32
131.51240196308936,1.0010692e+00,4.6094491e-04,0,1.408143e-03,383.13934,312.55242,1105,5.2593447e+03,2.4440820e+00,3.4708600e+02,1.0088471e-01,7.6917827e+03,3.5417016e+00,0,———,———,———,———,383.13934,3.6344884e-04,312.55242,2.5682544e-04,9.0776747e-03,-4.1715618e-02
131.5328366199028,1.0016155e+00,4.6129877e-04,0,1.409100e-03,383.13876,312.55305,1106,5.2646748e+03,2.4447544e+00,3.4733621e+02,1.0084715e-01,7.6959810e+03,3.5444205e+00,0,———,———,———,———,383.13876,3.6338420e-04,312.55305,2.5676336e-04,9.0881726e-03,-4.1500472e-02
131.55327117648994,1.0013910e+00,4.6126171e-04,0,1.410056e-03,383.13936,312.55250,1107,5.2636416e+03,2.4445834e+00,3.4727063e+02,1.0094051e-01,7.6942554e+03,3.5441358e+00,0,———,———,———,———,383.13936,3.6321729e-04,312.55250,2.5664648e-04,8.9672692e-03,-4.1346982e-02
131.57370563306904,1.0015259e+00,4.6140185e-04,0,1.411013e-03,383.13960,312.55255,1108,5.2641406e+03,2.4446614e+00,3.4706705e+02,1.0079391e-01,7.6952925e+03,3.5452125e+00,0,———,———,———,———,383.13960,3.6329808e-04,312.55255,2.5667142e-04,8.7374439e-03,-4.1336384e-02
131.59414018965617,1.0010363e+00,4.6126716e-04,0,1.411970e-03,383.13928,312.55247,1109,5.2606289e+03,2.4441779e+00,3.4729874e+02,1.0081837e-01,7.6915303e+03,3.5441775e+00,0,———,———,———,———,383.13928,3.6335408e-04,312.55247,2.5675929e-04,8.8381814e-03,-4.1683424e-02
131.61457484612765,1.0009363e+00,4.6132209e-04,0,1.412926e-03,383.13963,312.55317,1110,5.2595791e+03,2.4440963e+00,3.4763492e+02,1.0077314e-01,7.6907622e+03,3.5445995e+00,0,———,———,———,———,383.13963,3.6350836e-04,312.55317,2.5682620e-04,8.8688750e-03,-4.1656967e-02
131.6350093024812,1.0010699e+00,4.6145751e-04,0,1.413882e-03,383.14009,312.55249,1111,5.2601133e+03,2.4441595e+00,3.4743008e+02,1.0083368e-01,7.6917886e+03,3.5456400e+00,0,———,———,———,———,383.14009,3.6333539e-04,312.55249,2.5671266e-04,8.9300554e-03,-4.1637775e-02
131.6554438587118,1.0008240e+00,4.6129979e-04,0,1.414839e-03,383.13930,312.55247,1112,5.2581855e+03,2.4439130e+00,3.4746689e+02,1.0074146e-01,7.6898994e+03,3.5444283e+00,0,———,———,———,———,383.13930,3.6342334e-04,312.55247,2.5679090e-04,9.1093248e-03,-4.1698195e-02


In [32]:
test1[0].head()

time,flux,flux_err,quality,timecorr,centroid_col,centroid_row,cadenceno,sap_flux,sap_flux_err,sap_bkg,sap_bkg_err,pdcsap_flux,pdcsap_flux_err,sap_quality,psf_centr1,psf_centr1_err,psf_centr2,psf_centr2_err,mom_centr1,mom_centr1_err,mom_centr2,mom_centr2_err,pos_corr1,pos_corr2
,electron / s,electron / s,,d,pix,pix,,electron / s,electron / s,electron / s,electron / s,electron / s,electron / s,,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix
Time,float32,float32,int32,float32,float64,float64,int32,float32,float32,float32,float32,float32,float32,int32,float64,float32,float64,float32,float64,float32,float64,float32,float32,float32
131.51240196308936,7.6917827e+03,3.5417016e+00,0,1.408143e-03,383.13934,312.55242,1105,5.2593447e+03,2.4440820e+00,3.4708600e+02,1.0088471e-01,7.6917827e+03,3.5417016e+00,0,———,———,———,———,383.13934,3.6344884e-04,312.55242,2.5682544e-04,9.0776747e-03,-4.1715618e-02
131.5328366199028,7.6959810e+03,3.5444205e+00,0,1.409100e-03,383.13876,312.55305,1106,5.2646748e+03,2.4447544e+00,3.4733621e+02,1.0084715e-01,7.6959810e+03,3.5444205e+00,0,———,———,———,———,383.13876,3.6338420e-04,312.55305,2.5676336e-04,9.0881726e-03,-4.1500472e-02
131.55327117648994,7.6942554e+03,3.5441358e+00,0,1.410056e-03,383.13936,312.55250,1107,5.2636416e+03,2.4445834e+00,3.4727063e+02,1.0094051e-01,7.6942554e+03,3.5441358e+00,0,———,———,———,———,383.13936,3.6321729e-04,312.55250,2.5664648e-04,8.9672692e-03,-4.1346982e-02
131.57370563306904,7.6952925e+03,3.5452125e+00,0,1.411013e-03,383.13960,312.55255,1108,5.2641406e+03,2.4446614e+00,3.4706705e+02,1.0079391e-01,7.6952925e+03,3.5452125e+00,0,———,———,———,———,383.13960,3.6329808e-04,312.55255,2.5667142e-04,8.7374439e-03,-4.1336384e-02
131.59414018965617,7.6915303e+03,3.5441775e+00,0,1.411970e-03,383.13928,312.55247,1109,5.2606289e+03,2.4441779e+00,3.4729874e+02,1.0081837e-01,7.6915303e+03,3.5441775e+00,0,———,———,———,———,383.13928,3.6335408e-04,312.55247,2.5675929e-04,8.8381814e-03,-4.1683424e-02
